# <div style="text-align:center"><span style="color:blue"> TRABAJO FINAL DE LENGUAJES DE PROGRAMACION II</span> </div> 
 
 ## <div ><span style="color:blue"> Integrantes:</span> </div>
 1. Llalle Correa Danixa
 2. Matos Salazar Ruben
 3. Montoya Toribio Jenny Antonella
 4. Morales Morales Flavio Oscar
 
 <span style="color:green;"> Semestre: 2021-2</span>
 ### <div ><span style="color:blue"> Indicaciones </span> </div>
 - El trabajo es grupal (integrantes de grupos ya indicados)
 - Utilizar Jupyter notebook.
 - Trabajar en equipo vía GitHub entre los miembros de los grupos. (Sugerencia: el jefe de grupo puede crear el repositorio del proyecto e invitar a los demás integrantes a colaborar en el proyecto).
 
 ### <div ><span style="color:blue"> Procedimiento Referencial</span> </div>
 1. Descargar y explorar el archivo genes identificados con vecinos.xlsx compuesta por varias hojas las cuales tienen una columna con etiqueta: GenAbrev.
 2. Para cada nombre del gen en abreviado (GenAbrev), abrir la página https://www.uniprot.org y en el buscador solicitar la búsqueda de cada una de la forma siguiente: colocar el nombre del gen y la palabra Bos Taurus. Abajo se muestra para UBA52 Bos taurus.
 3. Se mostrará una lista de genes con el nombre buscado. Extraer información de entry y entry name.
 4. Seleccionar la primera opción del gen de la primera columna “entry” (clickear), la cual llevará a la siguiente pantalla y de la cual extraer la información de Protein, Gene,Organism y Status.
 5. Extarer la infomación que corresponde a: GO - Biological process.

In [1]:
######################### IMPORTANTE #################################

# SE DEBE TENER AL CODIGO Y AL EXCEL VACIO EN UNA MISMA CARPETA
# (AMBOS EN LA MISMA CARPETA)

In [6]:
from bs4 import BeautifulSoup   # importa la libreria Beautifulsoup
import requests                 # importa libreria requests
import pandas as pd             # importa la libreria pandas
import openpyxl                 # importa la libreria openpyxl
import time                     # importa la libreria time

archivo_excel_vacio = pd.read_excel("/TablaVacia.xlsx",header=1,index_col=0) # leer excel de cabecera fila 1 y indice columna 0
lista_genes_abreviados=list(archivo_excel_vacio["GEN ABREV"]) # importamos la columna que tiene los genes abreviados y los introducimos a una lista
m=3     # se incia en la fila 3
for i in lista_genes_abreviados: # lee cada abreviación del gen
    time.sleep(1)        # es el espacio del tiempo que hay entre cada ejecución del for 
    print(m-2,":",i)     # imprime la fila y el gen que se esta procesando
    URL=f"https://www.uniprot.org/uniprot/?query={i.upper()}+bos+taurus&sort=score" # introducimos el url de busqueda de la pagina requerida
    page=requests.get(URL)  # conección a la pagina
    soup=BeautifulSoup(page.content,"html.parser")  # almacena el codigo html

    link_entry=soup.find_all("td",class_="entryID") # lista de entradas que aparacen en el resultado de la busqueda
    indice=[]                  # genera una lista vacia
    for P in link_entry:       # para cada entrada que aparece en el resultado de busqueda se almacena en la lista indice 
        indice.append(P.text)  # añade los resultados a la lista vacia
    try:   # si hay un resultado en el indice va intentar ejecutar el siguiente codigo
        FINAL=indice[0]     # selecciona el primer resultado
        url=f"https://www.uniprot.org/uniprot/{FINAL}" # introduce el url del primer resultado
        page=requests.get(url)  # conección a la pagina
        soup=BeautifulSoup(page.content,"html.parser") # almacena el codigo html
               
        #ENTRY Y ENTRY NAME 
        entry_name=soup.find_all("h2",class_="page-title")  # busca todos los elementos que tengan etiqueta h2 
        for Q in entry_name:   # limpia los resultados y extrae unicamente los textos
            entry1=Q.text      # extrae unicamente los textos
        entry1=entry1.split()  # separa los textos encontrados
        ENTRY=entry1[2]        # selecciona el elemento número 2 de los textos
        ENTRY_NAME="".join(list(entry1[3])[1:len(list(entry1[3]))-1])   # introduce lo encontrado anteriormente y lo selecciona como entry_name

        #PROTEINA
        protein=soup.find_all("h1",property="name")   # busca todos los elementos que tenga etiqueta h1
        for Q in protein:     # limpia los resultados  
            PROTEIN=Q.text    # extrae unicamente los textos

        #GEN
        gen_abrev=soup.find_all("div",class_='entry-overview-content',id="content-gene") # busca todos los elementos que contienen id "content-gene", con etiqueta div.
        for Q in gen_abrev:     # limpia los resultados 
            GEN_ABREV=Q.text    # extrae unicamente los textos

        #ORGANISMO
        organismo=soup.find_all("div",class_='entry-overview-content',id="content-organism") # busca todos los elementos que contienen id "content-organism", con etiqueta div.
        for Q in organismo:  # limpia los resultados
            ORGANISMO=Q.text # extrae unicamente los textos

        #STATUS
        status=soup.find_all("span",class_='context-help bin-score tooltipped-click') # busca todos los elementos, con etiqueta span.
        for Q in status:       # limpia los resultados
            STATUS_PRE=Q.text  # extrae unicamente los textos
        STATUS=STATUS_PRE.split("\n")[0].split(":")[1] # selecciona el status de toda esa respuesta 

        #GO BIOLOGICAL PROCESS
        bio_process=soup.find_all("a",onclick="window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');") # busca todos los elementos, con etiqueta a
        BIO_PROCESS=[]                 # crea una lista
        for Q in bio_process:          # limpia los resultados
            BIO_PROCESS.append(Q.text) # selecciona los textos y los añade a la lista vacia
        BIO_PROCESS=str(BIO_PROCESS)   # vuelve toda la lista en una cadena 
        
        
    except:               # si es que no hay resultado en la busqueda dejara en blanco las celdas de los siguientes atributos
        ENTRY=""      
        ENTRY_NAME=""
        PROTEIN=""
        GEN_ABREV=""
        ORGANISMO=""
        STATUS=""
        BIO_PROCESS=""
        
        
    excel=openpyxl.load_workbook('/TablaVacia.xlsx')   # abre el excel para completar los datos obtenidos en las columnas
    sheet=excel['Hoja1']                               # hoja 1 del excel
    sheet.cell(row=m,column=3).value = ENTRY           # llena los valores 
    sheet.cell(row=m,column=4).value = ENTRY_NAME
    sheet.cell(row=m,column=5).value = PROTEIN
    sheet.cell(row=m,column=6).value = GEN_ABREV
    sheet.cell(row=m,column=7).value = ORGANISMO
    sheet.cell(row=m,column=8).value = STATUS
    sheet.cell(row=m,column=9).value = BIO_PROCESS
    excel.save('/TablaVacia.xlsx')                     # guarda el excel con los datos obtenidos               
    m=m+1                                              # pasa a la siguiente fila

1 : TRNAG-UCC
2 : ROBO1
3 : TRNAC-GCA
4 : TRNAC-ACA
5 : MIR2286
6 : LOC101910021
7 : LOC112447295
8 : CD47
9 : IFT57
10 : HHLA2
11 : MYH15
12 : CIP2A
13 : DZIP3
14 : TRAT1
15 : MORC1
16 : C1H3orf85
17 : LOC112448290
18 : LOC101903052
19 : DPPA2
20 : DPPA4
21 : LOC100337136
22 : LOC783844
23 : RTP4
24 : MASP1
25 : RTP1
26 : LOC100847189
27 : ST6GAL1
28 : MIR2285K-3
29 : ADIPOQ
30 : RFC4
31 : EIF4A2
32 : MIR1248-1
33 : KNG1
34 : HRG
35 : FETUB
36 : AHSG
37 : LOC101906884
38 : DNAJB11
39 : TBCCD1
40 : CRYGS
41 : DGKG
42 : ETV5
43 : LOC616971
44 : TRA2B
45 : TRNAE-UUC
46 : IGF2BP2
47 : SENP2
48 : LIPH
49 : TMEM41A
50 : MAP3K13
51 : EHHADH
52 : EPHB1
53 : KY
54 : CEP63
55 : ANAPC13
56 : AMOTL2
57 : RYK
58 : SLCO2A1
59 : RAB6B
60 : TF
61 : LOC525947
62 : TOPBP1
63 : RIPK4
64 : PRDM15
65 : C2CD2
66 : ZBTB21
67 : UMODL1
68 : ABCG1
69 : TFF3
70 : TFF2
71 : TFF1
72 : TMPRSS3
73 : UBASH3A
74 : RSPH1
75 : SLC37A1
76 : PDE9A
77 : WDR4
78 : NDUFV3
79 : PKNOX1
80 : CBS
81 : U2AF1
82 : CRYAA
83 : SIK1

ConnectionError: ignored